In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
from keras.models import Sequential
import tensorflow as tf
tf.python.control_flow_ops = tf

import lstm, time #helper libraries
import numpy as np
from sklearn.metrics import mean_squared_error
import math

from pprint import pprint
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (18, 12)

In [ ]:
#Step 1 Load Data #load_data(filename, seq_len, normalise_window)
look_back = 50
X_train, y_train, X_test, y_test = lstm.load_data('sp500.csv', look_back, True)

print(X_train)
print(X_test)

In [ ]:
#Step 2 Build Model
batch_size = 1
epochs=1
model = Sequential()

model.add(LSTM(50,batch_input_shape=(batch_size, look_back, 1),return_sequences=True))#, dropout_W=0.2))
model.add(Dropout(0.2))

model.add(LSTM(100,return_sequences=False)) #, dropout_W=0.2))
model.add(Dropout(0.2))

model.add(Dense(
    output_dim=1))
model.add(Activation('linear'))

start = time.time()
model.compile(loss='mse', optimizer='rmsprop')
print 'compilation time : ', time.time() - start

In [ ]:
#Step 3 Train the model
model.fit(
    X_train,
    y_train,
    batch_size=batch_size,
    nb_epoch=epochs,
    validation_split=0.05)

In [ ]:
#Step 4 - Plot the predictions!
predictions = lstm.predict_sequences_multiple(model, X_test, batch_size, batch_size)
#lstm.plot_results_multiple(predictions, y_test, batch_size)


# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)

# Add labels
plt.ylabel('Price USD - Normalized')
plt.xlabel('Trading Days')
# Plot actual and predicted close values

plt.plot(y_test,'#5F5F5F', label='Adjusted Close')
plt.plot(predictions, '#0C91AB', label='Predicted Close')

# Set title
ax.set_title('Trading vs Prediction')
ax.legend(loc='upper left')

plt.show()

In [ ]:
preds = np.array(predictions).flatten()
#print(preds)
# The mean squared error
testScore = mean_squared_error(y_test, preds)
print('Test Score: %.4f MSE (%.4f RMSE)' % (testScore, math.sqrt(testScore)))

No Dropout: Test Score: 0.0003 MSE (0.0173 RMSE)

Dropout 20%: Test Score: 0.0002 MSE (0.0124 RMSE)